In [1]:
import json
import pennylane as qml
import pennylane.numpy as np

In [52]:
# Welcome to the QHack 2023 coding challenges!
def add_numbers(x, y):
    """This function adds two numbers together.

    Args:
        x (float): A number.
        y (float): A number.

    Returns:
        (float): The result of adding x and y.
    """
    
    dev = qml.device('default.qubit', wires=4, shots=1)
    
    @qml.qnode(dev)
    def one_bit_adder(x_val, y_val, c_in_val=0, c_out_val=0):
        x, y, c_in_sum_out, c_out = 0, 1, 2, 3
        if x_val: qml.PauliX(x)
        if y_val: qml.PauliX(y)
        if c_in_val: qml.PauliX(c_in_sum_out)
        if c_out_val: qml.PauliX(c_out)
        
        # Implement adder circ
        qml.Toffoli(wires=[x, y, c_out])
        qml.CNOT(wires=[x, y])
        qml.Toffoli(wires=[y, c_in_sum_out, c_out])
        qml.CNOT(wires=[y, c_in_sum_out])
        qml.CNOT(wires=[x, y])
        
        return qml.sample()
    
    def twos_comp(val, bits):
        if (val & (1 << (bits - 1))) != 0:
            val = val - (1 << bits)
        return val

    # Encode in binary format x and y with error
    error = 1000
    x_trunc = int(x * error)
    y_trunc = int(y * error)

    num_bits = max(len(str(bin(n))) for n in [x_trunc, y_trunc]) - 2
    x_two_cmp = twos_comp(x_trunc, num_bits)
    y_two_cmp = twos_comp(y_trunc, num_bits)

    x_bits = [(x_two_cmp >> bit) & 1 for bit in range(num_bits - 1, -1, -1)]
    y_bits = [(y_two_cmp >> bit) & 1 for bit in range(num_bits - 1, -1, -1)]
    
    c_in_sum_out = 0
    c_out = 0
    result = ''
    for x_i, y_i in zip(x_bits[::-1], y_bits[::-1]):
        _, _, c_in_sum_out, c_out = one_bit_adder(x_i, y_i, c_in_sum_out, c_out)
        result += str(c_in_sum_out)
        c_in_sum_out = int(c_out)
        c_out = 0
    # Add carry from last bit sum
    if x > 0 and y > 0:
        result += str(c_in_sum_out)
    
    # Decode from binary back to float
    return int(result[::-1], base=2) / error


In [53]:
# These functions are responsible for testing the solution.

def run(test_case_input: str) -> str:
    x, y = json.loads(test_case_input)
    result = add_numbers(x, y)
    return str(result)

def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(
        solution_output, expected_output, rtol=1e-4
    ), "Your addition function isn't quite right!"


In [54]:
test_cases = [['[0.2, -0.2]', '0.0']]

In [55]:
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[0.2, -0.2]'...
Correct!
